<a href="https://colab.research.google.com/github/jhyoo78/jhyoo78/blob/main/log_w2v_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files     # Colab에서 local file 읽어 들이는 라이브러리
uploaded = files.upload()

In [ ]:
import numpy as np
import nltk
nltk.download('punkt')   # ‘punkt=문장 tokenizer’를 다운로드
import urllib.request   # 아나콘다에 urllib3 만 있음
#import zipfile           # zipfile36만 있음, 파이선이 3.6 버전이어야 함
from lxml import etree   
import re                # 파이썬 정규표현을 위한 표준 라이브러리임
from nltk.tokenize import word_tokenize, sent_tokenize
import pandas as pd      # opencv에 conda install로 설치할 것
from gensim.models import Word2Vec     # opencv에  설치할 것
import matplotlib.pyplot as plt

In [ ]:
token_text = []     # 또는 list()

f = open('test.txt', 'r')
lines = f.readlines()
for line in lines:

    sent_text = sent_tokenize(line)
    #print(sent_text)
   
    # 각 문장에 대해서 구두점을 제거하고, 대문자를 소문자로 변환.
    normalized_text = []
    
    for string in sent_text:
        tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
        normalized_text.append(tokens)
        #print(normalized_text)
        
    # 각 문장에 대해서 NLTK를 이용하여 단어 토큰화를 수행.
    result = [word_tokenize(sentence) for sentence in normalized_text]
    result_arr= np.array(result).reshape(-1)  # List에서는 reshpe()를 사용 못하므로 array로 변환하여 처리함. reshape(-1)은 2차원 배열에서 1차원 배열만 출력함. 즉 [[ ]] --> [ ]
    #print(type(result_arr))  # ==> <class 'numpy.ndarray'>로 출력됨
    result_list=result_arr.tolist()   # array를 list 로 다시 변환
    #print("result=", result)
    #print("result_arr=", result_arr)     # [  ]
    token_text.append(result_list)        # token_text=[] 에 append()
    
#print('총 샘플의 개수 : {}'.format(len(token_text)))
print(type(token_text))
print("tokenized_text=", token_text)

for line in token_text[:3]:    # 앞의 3줄만 출력해 봄.
    print(line)

#print("tokenized text=", token_text)
#print(token_text[0][:])

f.close()

In [ ]:
model = Word2Vec(sentences=token_text, size=10, window=5, min_count=2, workers=4, iter=2, sg=0)   # worker== 사용할 CPU core의 개수
print("model=", model.wv)
print("wor2vector=", model.wv['errorh'])     # 특정 단어를 size 차원의 벡터로 변환하기

model_result = model.wv.most_similar("errorh", topn=10)   # 다른 유사한 단어(상위 10 개)와 유사도 계산
#model_result2 = model.wv.similarity(w1="failed", w2= "out")    # 2개 단어 간의 유사도 계산
print("model_result=", model_result)
#print("model_result2=", model_result2)
print('')


'''------------ 학습 결과 그리기 ==> https://m.blog.naver.com/PostView.naver?isHttpsRedirect=true&blogId=wideeyed&logNo=221349385092 ---------'''
def plot_2d_graph(vocabs, xs, ys):
    plt.figure(figsize=(10, 8))
    plt.scatter(xs, ys, marker = 'o')
    for i, v in enumerate(vocabs):
        plt.annotate(v, xy=(xs[i], ys[i]))

vocabs = model.wv.vocab.keys()
word_vectors_list=[model.wv[v] for v in vocabs]

'''--- 고차원 벡터를 시각화하기 위해서는 2~3차원 벡터로 축소하는 것이 필요하다. PCA는 대표적인 차원 축소 방식 중 하나임------'''
''' -------------------- PCA는 군집을 표현 못하니 t-SNE로 바꾸어 볼 것 ---------------------'''
from sklearn.decomposition import PCA

pca= PCA(n_components=2)
xys = pca.fit_transform(word_vectors_list)
print("xys=", xys)
print(len(xys))       # xys의 [] 길이는 단어의 개수와 같음, 예 : xys= [[-6.54008631e-02 -1.85528987e-02]....... [-2.234521e-02  3.39717996e-02]]
xs = xys[:, 0]
ys = xys[:, 1]
plot_2d_graph(vocabs, xs, ys)


In [ ]:
print(model.wv.vocab)   # 어휘사전은 단어의 빈도 순으로 출력 ???

In [ ]:
'''---------- 아래 코드는 https://stackoverflow.com/questions/31440803/how-to-fetch-vectors-for-a-word-list-with-word2vec 에서 인용--------- '''
''' ---------- 전체 vectors for word list 를 출력 ---------------'''
my_dict = dict({})
for idx, key in enumerate(model.wv.vocab):
    my_dict[key] = model.wv[key]
    # Or my_dict[key] = model.wv.get_vector(key)
    # Or my_dict[key] = model.wv.word_vec(key, use_norm=False)

#%store my_dict > filename.txt   # 파일로 저장하기

In [ ]:
print(my_dict)

In [ ]:
# 단어의 수
len(model.wv.vocab)

In [ ]:
print(model.wv.vocab)
model.wv.get_vector('log')

In [ ]:
# 학습한 모델 저장 및 불러오기....
model.save(file name)
>>> model = Word2Vec.load(file name)  # you can continue training with the loaded model!

In [ ]:
f = open('test.txt', 'r')
file_read = f.read()   # 줄단위는 read_limes(), 전체는 read()
#print(file_read)

sent_text = sent_tokenize(file_read)
print(sent_text)   # 전체를 하나의 벡터로 표시하고 \n으로 문장을 구분함 ['a b c...\n a ab c ...']

# 각 문장에 대해서 구두점을 제거하고, 대문자를 소문자로 변환.
normalized_text = []
    
for string in sent_text:
    tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
    normalized_text.append(tokens)
    print(normalized_text)

# 각 문장에 대해서 NLTK를 이용하여 단어 토큰화를 수행.
result = [word_tokenize(sentence) for sentence in normalized_text]
print(result)
#print('총 샘플의 개수 : {}'.format(len(result)))  # ==> 코드 오류

#for line in result[0:3]:
#    print(line)

f.close()


In [ ]:
import numpy as np

x = np.arange(12)  # ==> [0,1,2...11]
x = x.reshape(3,4)

print(x.reshape(-1,1))   # 1차원 배열의 원소를 1개로

print(x.reshape(-1,2))
print(x.reshape(-1,3)) # 1차원 배열의 원소를 3개로

x.reshape(-1)          # 1차원으로 출력